# 绘制 Mandelbrot 集

[Mandelbrot 集合](https://en.wikipedia.org/wiki/Mandelbrot_set)是复平面上所有使得迭代序列 
$$f_c(0),f_c(f_c(0)),f_c(f_c(f_c(0))),\ldots,$$
始终保持有界的那些复数 $c$ 组成的集合，其中 $f_c(z)=z^2+c$。

为了绘制 Mandelbrot 集，我们可以将图像的每个像素对应到复平面上一点 $c$，然后计算上面序列的前若干项是否不超过一定的范围，并根据结果进行染色。

In [ ]:
import taichi as ti
from taichi.math import cmul, dot, log2, vec2, vec3
ti.init(arch=ti.cpu)

width, height = 800, 640
pixels = ti.Vector.field(3, float, shape=(width, height))

@ti.func
def setcolor(z, i):
    v = log2(i + 1 - log2(log2(z.norm()))) / 5
    col = vec3(0)
    if v < 1.0:
        col = vec3(v**4, v**2.5, v)
    else:
        v = ti.max(0., 2 - v)
        col = vec3(v, v**1.5, v**3)
    return col

@ti.kernel
def render():
    for i, j in pixels:
        c = 2.0 * vec2(i, j) / height - vec2(1.8, 1)
        z = vec2(0)
        count = 0
        while count < 100 and dot(z, z) < 50:
            z = cmul(z, z) + c
            count += 1

        if count == 100:
            pixels[i, j] = [0, 0, 0]
        else:
            pixels[i, j] = setcolor(z, count)

render()

from PIL import Image
import IPython

img = Image.fromarray((255 * pixels.to_numpy()).astype('uint8')).transpose(Image.Transpose.TRANSPOSE)
IPython.display.display(img)